# Travaux Pratiques 

---------
# Recovering Secret Exponent in RSA (Modular Exponentiation Analysis) 

## WITH Advanced (Differential) Side Channel Analysis
---------

To provide efficient RSA operation in hardware chip manufacturers usually embed arithmetic coprocessors to compute efficient modular multiplications x × y mod n for long integers x, y and n.

See slide ``Modular Arithmetic Methods``from the lesson slides.


Many techniques exist: interleaved multiplication- reduction with Knuth, Barrett, Montgomery, Sedlack or Quisquater methods [Dhe98].


<img src='images/Mod_expo_methods.png' style='width: 900px; float:center'>


<img src='images/Mod_mult_methods.png' style='width: 700px; float:center'>



Montgomery introduced in [Mon85] an efficient algorithm named Montgomery Modular Multiplication. 


<img src='images/MontMul.png' style='width: 700px; float:center'>


Modular exponentiation is the most time-consuming operation of RSA primitives. It is then essential to use an efficient method for exponentiation. Alg. 2.3. below, based on MontMul, gives the Montgomery exponentiation algorithm and is particularly suited for embedded RSA implementations.

<img src='images/MontModExp.png' style='width: 700px; float:center'>


**References**


[Dhe98] Dhem, J.-F.: Design of an efficient public-key cryptographic library for RISC-based smart cards. PhD thesis, Université catholique de Louvain, Louvain (1998)

[KAK96] Koc, C ̧ K., Acar, T., Kaliski, B.-S.: Analysing and comparing Montgomery multiplication algorithms. IEEE Micro 16(3), 26–33 (1996)

[Mon85] Montgomery, P.L.: Modular multiplication without trial division. Mathematics of Computation 44(170), 519–521 (1985)

---------


In [ ]:
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
import estraces
import scared
import numpy as np

In [ ]:
ths = estraces.read_ths_from_ets_file('../SideSCA-Traces-Public/RSA_SaM_traces.ets')
print(ths)

In [ ]:
vhex = np.vectorize(hex)

In [ ]:
# some parameters
modulus_bit_length = 1024
n_mod = 0xB828D7D0131A42A9FF63041DB16306639646E436367526638355881B831E7FAF33AE61EF6FC6E8961F4D6988A7F7A95FE9AC065E9A0C39595867DFE2ABFF9FA2C7876422AD5A40DEE4443EA7E019C32C9F6E172870CD7CA675AE705CA9148221506DA849DDA38A1B5701DDC554297F457A25A9FE5FAC2008B5D2FCA1C5BC281F
e_pub = 3

In [ ]:
ths.plaintext

Print the plaintext values in hexadecimal representation

In [ ]:
print(vhex(ths[0].plaintext))

How to get the plaintext integer value from bytes with simple operation

In [ ]:
plaintext_value = int.from_bytes(ths.plaintext[0], byteorder='big')
hex(plaintext_value)

What is the exponent value in ths ?

In [ ]:
exponent_value = int.from_bytes(ths.exponent[0], byteorder='big')
hex(exponent_value)

---------
---------

<img src='images/DoIt.png' style='width: 100px'>

## The traces below contain the beginning of an RSA exponentiation for the first bytes of exponent. 

## ---> How to recover the secret exponent bytes from traces?

---------
---------

In [ ]:
print("Length of each exponentiation trace is " + str(len(ths[0].samples))+" points")

### 🫵 Your turn: plot the trace

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize']=(18,6)

In [ ]:
...

### 🫵 Your turn:can you recover the secret exponent with SPA ?

### 🫵 Your turn: what can you test to identify where the multiplication operations are ?

-----

<img src='images/DoIt.png' style='width: 100px'>

## Using Reverse Side-Channel Analysis on which known metadata?

#### What is the value manipulated during a multiplication?

#### What about reverse analysis on plaintext ?


In [ ]:
@scared.reverse_selection_function
...

In [ ]:
container = scared.Container(ths)

nb_words parameters is used to define the base for representation of the value used for correlation.

* nb_words = 1 means the representation of the value used for model is in 1 byte representation --> 8-bit base representation for the value
* nb_words = 2 means the representation of the value used for model is in 2 byte representation --> 16-bit base representation for the value
* nb_words = 4 means the representation of the value used for model is in 4 byte representation --> 32-bit base representation for the value

In [ ]:
reverse_plaintext = ...
reverse_plaintext.run(container)

In [ ]:
reverse_plaintext.results.shape

In [ ]:
print(str(modulus_bit_length//8)+" bytes")

### Plot results

In [ ]:
...

#### Do we have exploitable information in reverse correlation traces ?

A hardware multiplier $a \times b$ to be efficient manipulates values in $b$ base.
Bigger is $b$: $2^^{16}, 2^^{32}, 2^^{64}$, faster is the multiplication.

Therefore to improve correlation we have to correlate with plaintext values in exact $b$ base representation.

* If $b = 2^^{8}$ then use plaintext in 8-bit decomposition: bytes
* If $b = 2^^{16}$ then use plaintext in 16-bit words decomposition: 2-bytes
* If $b = 2^^{32}$ then use plaintext in 32-bit words decomposition: 4-bytes

Let's try the reverse with other base $b$

In [ ]:
reverse_plaintext = ...
reverse_plaintext.run(container)

In [ ]:
reverse_plaintext.results.shape

In [ ]:
print(str(modulus_bit_length//16)+" 16-bit words")

### Plot results

In [ ]:
...

### Plot results

In [ ]:
plt.plot()
plt.title('Reverse base 32-bit for plaintext representation')
plt.show()

#### We know the plaintext is not randomized ? 

### So why do we have no correlation with the traces ?

-------------

#### If the modular mutiplication is Montgomery modular multiplication, then what is the value manipulated during modular multiplications ? Is is $a \times m$ or ?

### THEN test the reverse analysis with the plaintext in Montgomery representation as it should be manipulated by the hardware multiplier:  $a \times r \times m \bmod n$

Remember:

<img src='images/MontModExp.png' style='width: 700px; float:center'>


In [ ]:
r = ...
hex(r)

----------
### So we perform the reverse analysis with ???

In [ ]:
def identity_plaintext_Montgomery(plaintext, n, bitlen):
    montgomery_cst = ...
    ...
    sign_array = np.zeros((len(plaintext), modulus_base_len), dtype=np.uint8)
 
    ...
    ...
    

    return sign_array

In [ ]:
@scared.reverse_selection_function
def identity_plaintext_Montgomery_representation(plaintext):
    return identity_plaintext_Montgomery(plaintext, n_mod, modulus_bit_length)

In [ ]:
reverse_plaintext = scared.CPAReverse(selection_function=identity_plaintext_Montgomery_representation, model = scared.HammingWeight())
reverse_plaintext.run(container)

In [ ]:
reverse_plaintext.results.shape

### Plot results

In [ ]:
...

#### TRY it with 16-bit base

In [ ]:
...

In [ ]:
...

### Plot results

In [ ]:
...

#### TRY it with 32-bit base

In [ ]:
...

In [ ]:
...

### Plot results

In [ ]:
...

### --> CORRECT base used is ???????-bit as we see higher correlation.

------------------

### 🫵 Your turn: can you recover with SPA on this correlation trace the secret exponent ?

???

-------------------

### 🫵 Your turn: If not then looking at the trace, what do you conclude on the algorithm used to perform the modular exponentiation?

???

-----

## ATTACK OR NO ATTACK SUCCESS HERE ???




-----

<img src='images/DoIt.png' style='width: 100px'>

## How can we recover the secret exponent using correlation analysis ?
## What would be "DPA" like attack to do that ?

--------

### WE modify our function to reverse ???

In [ ]:
def 



    return sign_array

In [ ]:
guess = ...

In [ ]:
@scared.reverse_selection_function
def ...(plaintext):
    return ...

In [ ]:
reverse_expo_guess = scared.CPAReverse(selection_function=..., model = scared.HammingWeight(nb_words=2))
reverse_expo_guess.run(container)

In [ ]:
reverse_expo_guess.results.shape

### Plot results

In [ ]:
plt.plot(reverse_expo_guess.results[0].T)
plt.title('Reverse trace for plaintext^'+str(guess)+' mod n')
plt.show()

We have recovered The two first bits of the secret exponent.

d = 0bxx... = 0x...

Let's continue: next exponent bit is either 1 or 0:

* d= 0bxx0 and $plaintext^d \bmod n$ is computed
* d= 0bxx1 and $plaintext^d \bmod n$ is computed

In [ ]:
guess = 0b...

In [ ]:
@scared.reverse_selection_function
def ...(plaintext):
    return ...

In [ ]:
reverse_expo_guess = scared.CPAReverse(selection_function=..., model = scared.HammingWeight(nb_words=2))
reverse_expo_guess.run(container)

In [ ]:
plt.plot(reverse_expo_guess.results[0].T)
plt.title('Reverse trace for plaintext^'+str(guess)+' mod n')
plt.show()

#### Secret exponent known is now: 0bxxx...

#### let's continue with fourth bit of secret exponent ...

In [ ]:
guess = 0b...

### Then let's continue bit per bit .. or 2-bit per 2-bit (4 guesses) ... or nibble per nibble (16 guesses) ... or byte per byte (256 guesses) .... etc

In [ ]:
@scared.reverse_selection_function
def ...(plaintext):
    return 

### Plot results

In [ ]:
%matplotlib inline

plt.rcParams['figure.figsize']=(20,8)
plt.plot(reverse_expo_guess.results[31].T)
plt.title('Plaintext Reverse')
plt.show()

# Conclusion

* Attacking modular integer operations it is important to identify the proper modular arithmetic used for operations

* Regular algorithm like Square & Multiply algorithms (or Ladder) can be defeated with DPA like analysis

* Blinding on message and/or exponent is required to counterfeit such attacks

* Be aware more complex attack like horizontal attacks exist to apply such DPA on single trace attacks